In [ ]:
!ls

In [ ]:
!pip install deepcut --user

In [ ]:
!pip install flask_ngrok --user

In [1]:
import deepcut
import pandas as pd
import numpy as np
from itertools import chain
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import joblib
import pickle
import scipy.sparse as sp

In [2]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
import json

In [3]:
data = pd.read_excel("Category.xlsx")
data

,Category
0,หลักสูตร
1,ฝึกงาน
2,ลงทะเบียนเรียน
3,การรับเข้านักศึกษา
4,ทุนการศึกษา
5,คำถามทั่วไป


In [4]:
#Load File
with open('token_text_category.data', 'rb') as filehandle:
    # read the data as binary data stream
    tokenized_texts = pickle.load(filehandle)

In [5]:
tokenized_texts

0      [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์,  , มี, ...
1      [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์, หลักสูต...
2      [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์, หลักสูต...
3      [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์, หลักสูต...
4      [วิศวะ, คอม, /, วิศวกรรม, คอมพิวเตอร์, หลักสูต...
                             ...                        
172     [การ, สมัคร, กู้ยืม, เรียน,  , ทำ, อย่าง, ไร, ?]
173             [ทุน, ของ, ภาค, วิชา, มี, อะไร, บ้าง, ？]
174       [ทุน, เพชรพระจอมเกล้าภาค, วิชารับ, กี่, คน, ？]
175                       [ทุน, จ้าง, งาน, คือ, อะไร, ？]
176    [ถ้า, อยาก, ไป, เรียน, ต่อต่าง, และ, ขอ, ทุน, ...
Name: CPE_questions, Length: 177, dtype: object

In [6]:
def text_to_bow(tokenized_text, vocabulary_):
    n_doc = len(tokenized_text)
    values, row_indices, col_indices = [], [], []
    for r, tokens in enumerate(tokenized_text):
        feature = {}
        for token in tokens:
            word_index = vocabulary_.get(token)
            if word_index is not None:
                if word_index not in feature.keys():
                    feature[word_index] = 1
                else:
                    feature[word_index] += 1
        for c, v in feature.items():
            values.append(v)
            row_indices.append(r)
            col_indices.append(c)
        #print(feature)

    # document-term matrix in sparse CSR format
    X = sp.csr_matrix((values, (row_indices, col_indices)),
                      shape=(n_doc, len(vocabulary_)))
    return X

vocabulary_ = {v: k for k, v in enumerate(set(chain.from_iterable(tokenized_texts)))}
X = text_to_bow(tokenized_texts, vocabulary_)

In [7]:
transformer = TfidfTransformer()
svd_model = TruncatedSVD(n_components=100,
                         algorithm='arpack', n_iter=100)
X_tfidf = transformer.fit_transform(X)
X_svd = svd_model.fit_transform(X_tfidf)

In [8]:
tag = pd.get_dummies(data.Category).columns

In [9]:
#Lib
import joblib

#Load Model
logist_models = joblib.load("category_model.pkl")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [10]:
logist_models

[LogisticRegression(),
 LogisticRegression(),
 LogisticRegression(),
 LogisticRegression(),
 LogisticRegression(),
 LogisticRegression()]

In [11]:
y_pred = np.argmax(np.vstack([model.predict_proba(X_svd)[:, 1] for model in logist_models]).T, axis=1)
y_pred = np.array([tag[yi] for yi in y_pred])
y_true = data.Category.values
print(tag[0:6])

Index(['การรับเข้านักศึกษา', 'คำถามทั่วไป', 'ทุนการศึกษา', 'ฝึกงาน',
       'ลงทะเบียนเรียน', 'หลักสูตร'],
      dtype='object')


# use for deploy

In [12]:
app = Flask(__name__)
@app.route('/category', methods=['POST', 'GET'])
def home():
  if request.method == 'POST':
    data = request.get_json()
    print(data["inputQuestion"])
    text = data["inputQuestion"]
    tokenized_text = deepcut.tokenize(text)
    x = text_to_bow([tokenized_text], vocabulary_)
    x_tfidf = transformer.transform(x)
    x_svd = svd_model.transform(x_tfidf)
    pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
    print(list(zip(tag, pred)))
    predict_category = max(list(zip(tag, pred)))
    max_value = 0
    max_category = ''
    pred_results = list(zip(tag, pred))
    for pred_result in pred_results:
      # print(pred_result)
      if pred_result[1] > max_value:
        max_value = pred_result[1]
        max_category = pred_result[0]
    print(max_category, max_value)
    value = {
      "category": max_category,
      "accuracy": max_value
    }
    return json.dumps(value, ensure_ascii=False).encode('utf8')
  else:
    return "Hello. I am alive!"

app.run(port=5000, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


วิศวคอมมีหลักสูตรอะไรบ้าง


127.0.0.1 - - [13/May/2021 18:18:39] "POST /category HTTP/1.1" 200 -


[('การรับเข้านักศึกษา', 0.13460088729294772), ('คำถามทั่วไป', 0.20348234745059746), ('ทุนการศึกษา', 0.05078302803358382), ('ฝึกงาน', 0.02942253549775357), ('ลงทะเบียนเรียน', 0.0830843008910698), ('หลักสูตร', 0.41979603189236486)]
หลักสูตร 0.41979603189236486


# example of api (json)

In [ ]:
{
"inputQuestion":"วิศวคอมมีหลักสูตรอะไรบ้าง"
}

In [ ]:
# text =  'วิศวคอมพิวเตอร์มีหลักสูตรอะไรบ้าง'
# tokenized_text = deepcut.tokenize(text)
# x = text_to_bow([tokenized_text], vocabulary_)
# x_tfidf = transformer.transform(x)
# x_svd = svd_model.transform(x_tfidf)
# pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
# print(list(zip(tag, pred)))
# predict_category = max(list(zip(tag, pred)))

In [ ]:
# max_value = 0
# max_category = ''
# pred_results = list(zip(tag, pred))
# for pred_result in pred_results:
#   # print(pred_result)
#   if pred_result[1] > max_value:
#     max_value = pred_result[1]
#     max_category = pred_result[0]
# print(max_category, max_value)

# flask with auto ngrok

In [ ]:
# app = Flask(__name__)
# run_with_ngrok(app)   #starts ngrok when the app is run
# @app.route('/', methods=['POST', 'GET'])
# def home():
#   if request.method == 'POST':
#     data = request.get_json()
#     print(data["inputQuestion"])
#     text = data["inputQuestion"]
#     tokenized_text = deepcut.tokenize(text)
#     x = text_to_bow([tokenized_text], vocabulary_)
#     x_tfidf = transformer.transform(x)
#     x_svd = svd_model.transform(x_tfidf)
#     pred = [model.predict_proba(x_svd.reshape(-1, 1).T).ravel()[1] for model in logist_models]
#     print(list(zip(tag, pred)))
#     predict_category = max(list(zip(tag, pred)))
#     max_value = 0
#     max_category = ''
#     pred_results = list(zip(tag, pred))
#     for pred_result in pred_results:
#       # print(pred_result)
#       if pred_result[1] > max_value:
#         max_value = pred_result[1]
#         max_category = pred_result[0]
#     print(max_category, max_value)
#     value = {
#       "category": max_category,
#       "accuracy": max_value
#     }
#     return json.dumps(value, ensure_ascii=False).encode('utf8')
#   else:
#     return "Hello. I am alive!"

# app.run()